#  Day 1: Depth Calibration with Pressure Sensor and LED Feedback
##  Step 0: Pair with Your Databot
Run the cell below to scan and save the Bluetooth address of your Databot. This only needs to be done once per computer.

```python
from databot import PyDatabot
PyDatabot.get_databot_address(force_address_read=True)
```

##  Step 1: Databot Configuration
You're going to enable the pressure sensor and set the LED to blue.

### `DatabotConfig()` Parameters:
- `pressure = True`: enables the atmospheric pressure sensor
- `led1 = DatabotLEDConfig(state, R, G, B)` sets LED1 with RGB values

In [ ]:
# ✅ DONE: Set up Databot configuration
from databot import DatabotConfig, DatabotLEDConfig, PyDatabot

c = DatabotConfig()
c.pressure = True
c.led1 = DatabotLEDConfig(True, 0, 0, 255)  # Blue
c.address = PyDatabot.get_databot_address()

### DatabotLEDConfig Format
`DatabotLEDConfig(state, R, G, B)` configures the LED behavior on the Databot.

**Parameters:**
- `state` (bool): `True` to turn the LED on, `False` to turn it off
- `R` (int): Red intensity (0 to 255)
- `G` (int): Green intensity (0 to 255)
- `B` (int): Blue intensity (0 to 255)

**Examples:**
- `DatabotLEDConfig(True, 255, 0, 0)` → Red LED on
- `DatabotLEDConfig(True, 0, 255, 0)` → Green LED on
- `DatabotLEDConfig(True, 0, 0, 255)` → Blue LED on
- `DatabotLEDConfig(False, 0, 0, 0)` → LED off

##  Step 2: Convert Pressure to Depth
Write a function that calculates depth from a pressure value in Pascals.
- Use sea level pressure = `101325`
- Use water density = `1000`
- Use gravity = `9.8`

In [ ]:
# 🧠 TODO: Complete this function
def calculate_depth(pressure):
    # Subtract sea level pressure
    adjusted = pressure - 101325
    # Convert to depth in meters
    depth = adjusted / (1000 * 9.8)  # ρ * g
    return depth

# Test your function
print(f"Test depth: {calculate_depth(102325):.2f} m")

##  Step 3: Write a Function to Set LED Based on Depth
Write a function that returns a `DatabotLEDConfig` color based on the depth:
- Blue for < 1.0 m
- Green for 1.0–2.5 m
- Red for > 2.5 m

In [ ]:
# 🧠 TODO: Fill in this LED decision function
def get_led_color(depth):
    if depth > 2.5:
        return DatabotLEDConfig(True, 255, 0, 0)  # Red
    elif depth > 1.0:
        return DatabotLEDConfig(True, 0, 255, 0)  # Green
    else:
        return DatabotLEDConfig(True, 0, 0, 255)  # Blue

# Try it
print(get_led_color(0.5))

##  Step 4: Create a Custom Logger
This class logs data from the Databot, calculates depth, sets the LED, and writes to a file.

In [ ]:
# ✅ DONE: Logger class using your functions
import json
from databot import PyDatabotSaveToFileDataCollector

class DepthFeedbackLogger(PyDatabotSaveToFileDataCollector):
    def process_databot_data(self, epoch, data):
        # Get pressure value
        pressure = float(data.get("pressure", 101325))
        # 🔧 Use your function to calculate depth
        depth = calculate_depth(pressure)
        data["depth"] = round(depth, 2)

        # 🔧 Set the LED based on depth
        self.databot_config.led1 = get_led_color(depth)

        # Save to file
        data['timestamp'] = epoch
        with open(self.file_path, 'a') as f:
            f.write(json.dumps(data) + "\n")

        print(f"Depth: {depth:.2f} m | Pressure: {pressure:.2f} Pa")
        self.record_number += 1
        if self.number_of_records_to_collect and self.record_number >= self.number_of_records_to_collect:
            raise Exception("Data collection complete")

##  Step 5: Start the Dive Logger
This collects 100 readings and stores them in a file.

In [ ]:
# ✅ DONE: Run logger
filename = "depth_log_day1.txt"
logger = DepthFeedbackLogger(c, filename, number_of_records_to_collect=100)
logger.run()

##  Step 6: Plot Your Dive Data
After the dive, load the data file and create a graph showing depth over time.

In [ ]:
# ✅ DONE: Plotting code
import matplotlib.pyplot as plt

depths = []
timestamps = []

with open(filename, 'r') as f:
    for line in f:
        record = json.loads(line)
        depths.append(record["depth"])
        timestamps.append(record["timestamp"])

plt.plot(timestamps, depths, label="Depth (m)")
plt.xlabel("Time (s)")
plt.ylabel("Depth (m)")
plt.title("ROV Dive Depth Log")
plt.grid(True)
plt.legend()
plt.show()